In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

def read_excel_data(file_path):
    # Reading Excel file using pandas
    df = pd.read_excel(file_path, engine='openpyxl')
    return df

def plot_data(df, player_name):
    time = df["Time_s_"]

    # Plotting Z force for both legs
    plt.figure()
    plt.plot(time, df["RawFz_R_N_"], label="Drive Leg Force (Z)")
    plt.plot(time, df["RawFz_L_N_"], label="Stride Leg Force (Z)")
    plt.title(f"Z Force Profile for {player_name}")
    plt.xlabel('Time (s)')
    plt.ylabel('Force (N)')
    plt.legend()

    # Plotting rate of force development for both legs in Z direction
    rate_of_force_R = np.gradient(df["RawFz_R_N_"].values)
    rate_of_force_L = np.gradient(df["RawFz_L_N_"].values)

    plt.figure()
    plt.plot(time, rate_of_force_R, label="Drive Leg RFD (Z)")
    plt.plot(time, rate_of_force_L, label="Stride Leg RFD (Z)")
    plt.title(f"Rate of Force Development in Z for {player_name}")
    plt.xlabel('Time (s)')
    plt.ylabel('Rate of Force (N/s)')
    plt.legend()

    # Plotting XYZ forces for both legs
    plt.figure()
    plt.plot(time, df["RawFx_L_N_"], label="Force X")
    plt.plot(time, df["RawFy_L_N_"], label="Force Y")
    plt.plot(time, df["RawFz_L_N_"], label="Force Z")
    plt.title(f"Force Profile (XYZ) for Stride Leg of {player_name}")
    plt.xlabel('Time (s)')
    plt.ylabel('Force (N)')
    plt.legend()

    plt.figure()
    plt.plot(time, df["RawFx_R_N_"], label="Force X")
    plt.plot(time, df["RawFy_R_N_"], label="Force Y")
    plt.plot(time, df["RawFz_R_N_"], label="Force Z")
    plt.title(f"Force Profile (XYZ) for Drive Leg of {player_name}")
    plt.xlabel('Time (s)')
    plt.ylabel('Force (N)')
    plt.legend()

    plt.show()

def synchronize_data_based_on_peak(all_data, column_name="RawFz_R_N_"):
    # Identify the index of the peak force in each dataset
    peak_indices = [df[column_name].idxmax() for df in all_data]

    # Find the maximum peak index (this will serve as the reference)
    max_peak_index = max(peak_indices)

    synchronized_data = []

    for df, peak_index in zip(all_data, peak_indices):
        # Calculate the offset required to align this dataset with the reference
        offset = max_peak_index - peak_index
        
        # Apply the offset
        if offset > 0:
            # If offset is positive, we prepend 'offset' number of NaN rows
            dummy_rows = pd.DataFrame({col: np.nan for col in df.columns}, index=[i for i in range(-offset, 0)])
            new_df = pd.concat([dummy_rows, df], axis=0).reset_index(drop=True)
        elif offset < 0:
            # If offset is negative, we remove 'offset' number of rows from the beginning
            new_df = df.iloc[-offset:].reset_index(drop=True)
        else:
            # If no offset is required, we use the dataframe as is
            new_df = df.reset_index(drop=True)

        synchronized_data.append(new_df)

    # The synchronized dataframes might have varying lengths now; we should adjust them to the same length
    max_length = max([len(df) for df in synchronized_data])

    for i, df in enumerate(synchronized_data):
        if len(df) < max_length:
            dummy_rows = pd.DataFrame({col: np.nan for col in df.columns}, index=[i for i in range(len(df), max_length)])
            synchronized_data[i] = pd.concat([df, dummy_rows], axis=0).reset_index(drop=True)

    return synchronized_data

def main():
    #base_path = "/Users/charlesarnold/Library/Mobile Documents/com~apple~CloudDocs/Documents/Shared Documents/LA Tech/2023/Hitting" # mac path
    base_path = r"Z:\LA Tech\2023\Hitting" # windows path
    

    player_name = input("Enter the name of the player: ")

    files = [f for f in os.listdir(base_path) if f.startswith(player_name)]

    all_data = []
    for file in files:
        file_path = os.path.join(base_path, file)
        df = read_excel_data(file_path)
        all_data.append(df)

    # Synchronize data based on peak force
    all_data = synchronize_data_based_on_peak(all_data)

    # Average data from multiple trials
    avg_data = sum(all_data) / len(all_data)

    plot_data(avg_data, player_name)

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'pandas'